In [15]:

import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import requests
import xml.etree.ElementTree as ET
import tensorflow as t
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords #Word Stop
from nltk.tokenize import word_tokenize #Tokenization & Word Stop"
from sklearn.naive_bayes import GaussianNB
#nltk.download('punkt_tab')


csv_file = '/home/abarovic/laneyREUTest/eclipse_platform.csv'
full_df = pd.read_csv(csv_file)

full_df.fillna(0,inplace=True)
length=len(full_df)
split_index=int(.8*length)
df=full_df.iloc[:split_index] #df=training set
#creating freeform textual data column
df['full_text_data']=df['Description']+df['Title']
#print(df['full_text_data'])
df['full_text_data']=df['full_text_data'].fillna('')
def is_duplicate(val):
    return 1 if val !=0.0 else 0
df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))


df.head()

X = df['full_text_data']  # Input features
y = df['isDuplicate']       # Target variable

n = len(df)
split_index = int(.8 * n)
train_dataset = df.iloc[:split_index]
validation_dataset = df.drop(train_dataset.index)

train_target = train_dataset['isDuplicate']  # target features
#topic_train_priority
train_feature = train_dataset['full_text_data']       # input variable
#topic_train_data
validation_target= validation_dataset['isDuplicate']  # feature features
#topic_validation_priority
validation_feature= validation_dataset['full_text_data']       # input variable
#topic_validation_data 

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['full_text_data'], df['isDuplicate'], test_size=0.25, random_state=42, stratify=df['isDuplicate'])
vectorizer = CountVectorizer(stop_words = 'english', tokenizer = word_tokenize, lowercase = True)

#train_feature = vectorizer.fit_transform(train_feature)
#validation_feature = vectorizer.transform(validation_feature)
train_feature = vectorizer.fit_transform(X_train)
validation_feature = vectorizer.transform(X_test)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['full_text_data'], df['isDuplicate'], test_size=0.25, random_state=42, stratify=df['isDuplicate'])


# Assuming train_feature, train_target, validation_feature, validation_target are already defined

# Slice the first 2000 rows for training
#train_array = train_feature[:2000].toarray()
#train_target = train_target[:2000]
train_array = train_feature[:2000].toarray()
train_target = train_target[:2000]
# Fit the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(train_array, train_target)

# Slice the first 200 rows for validation
validation_array = validation_feature[:200].toarray()
validation_target = validation_target[:200]

# Predict using the validation data
y_pred = gnb.predict(validation_array)

print("Validation target length:", len(validation_target))
print("y_pred length:", len(y_pred))
print(y_pred)
print(train_target)


# Assuming train_target and y_pred are already defined
# Assuming validation_target and validation_array are already defined from previous slicing

# Fit the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(train_array, train_target)

# Predict using the validation data
y_pred = gnb.predict(validation_array)

# Calculate and print the classification report
report = classification_report(validation_target, y_pred)
print(report)


/tmp/ipykernel_3390682/1979396960.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text_data']=df['Description']+df['Title']
/tmp/ipykernel_3390682/1979396960.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text_data']=df['full_text_data'].fillna('')
/tmp/ipykernel_3390682/1979396960.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

Validation target length: 200
y_pred length: 200
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0       0
1       0
2       0
3       0
4       0
       ..
1995    0
1996    1
1997    0
1998    0
1999    0
Name: isDuplicate, Length: 2000, dtype: int64
              precision    recall  f1-score   support

           0       0.84      0.98      0.90       167
           1       0.20      0.03      0.05        33

    accuracy                           0.82       200
   macro avg       0.52      0.50      0.48       200
weighted avg       0.73      0.82      0.76       200

